In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abecklas/fifa-world-cup")

print("Path to dataset files:", path)

c:\Users\Bruno\OneDrive\Machine learning\Azure AI custom\azure-sql-ai-chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 349k/349k [00:00<00:00, 605kB/s]

Extracting files...
Path to dataset files: C:\Users\Bruno\.cache\kagglehub\datasets\abecklas\fifa-world-cup\versions\5


# Connection to the server

In [4]:
import pyodbc
import csv
import os
from dotenv import load_dotenv
from dotenv import dotenv_values

# Azure SQL connection details
server_name = os.getenv("AZURE_SERVER_NAME")
database_name = os.getenv("DATABASE_NAME")
admin_username = os.getenv("DB_ADMIN_USERNAME")
admin_password = os.getenv("DB_ADMIN_PASSWORD")

connection_string = (
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER={server_name};"
        f"DATABASE={database_name};"
        f"UID={admin_username};"
        f"PWD={admin_password}"
    )

try:
    connection = pyodbc.connect(connection_string)
    print("Connection succesful!")    
    
except pyodbc.Error as e:
    print("Error connecting to the database:",e)

Connection succesful!


# Insert the csv data into the database

In [8]:
import pyodbc
import csv
import os
from dotenv import load_dotenv
from dotenv import dotenv_values

env = ".env"
if os.path.exists(env):
    load_dotenv(override=True)
    config= dotenv_values(env)

# Azure SQL connection details
server_name = os.getenv("AZURE_SERVER_NAME")
database_name = os.getenv("DATABASE_NAME")
admin_username = os.getenv("DB_ADMIN_USERNAME")
admin_password = os.getenv("DB_ADMIN_PASSWORD")

# File path to your CSV file
csv_file_path = "Datasets/WorldCupMatches.csv"

def load_csv_to_database():
    # Connect to the Azure SQL Database
    connection_string = (
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER={server_name};"
        f"DATABASE={database_name};"
        f"UID={admin_username};"
        f"PWD={admin_password}"
    )

    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()

    # Read CSV file and insert rows into the database
    with open(csv_file_path, mode="r", encoding="utf-8") as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip header row

        for row in csv_reader:
            cursor.execute("""
                INSERT INTO WorldCupMatches (
                    Year, Datetime, Stage, Stadium, City, HomeTeamName,
                    HomeTeamGoals, AwayTeamGoals, AwayTeamName, WinConditions,
                    Attendance, HalfTimeHomeGoals, HalfTimeAwayGoals,
                    Referee, Assistant1, Assistant2, RoundID, MatchID,
                    HomeTeamInitials, AwayTeamInitials
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, row)

    # Commit and close the connection
    connection.commit()
    connection.close()
    print("Data successfully inserted into WorldCupMatches table.")

if __name__ == "__main__":
    load_csv_to_database()


Data successfully inserted into WorldCupMatches table.
